In [1]:
import statisUtils as utils
import pandas as pd
import numpy as np
import random
from scipy.stats import loguniform, lognorm
from scipy import stats
import time
import math

import ABCSampler as abcSampler


In [2]:
# df = pd.read_csv('data/MEF_QC_all.csv', header=None) # gene x cells
df_cast = pd.read_csv('data/cast_genes_count.csv', header=None, index_col=[0]) # gene x cells
df_c57 = pd.read_csv('data/c57_genes_count.csv', header=None, index_col=[0]) # gene x cells

df = df_cast + df_c57


# data_all = np.array(df, dtype=int)
# data_all

In [3]:
df_cast

,1,2,3,4,5,6,7,8,9,10,...,673,674,675,676,677,678,679,680,681,682
0,,,,,,,,,,,,,,,,,,,,,
Mrpl15,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,1.0,2.0,0.0,0.0,3.0
4732440D04Rik,11.0,3.0,NaN,0.0,2.0,0.0,3.0,1.0,2.0,1.0,...,4.0,1.0,1.0,5.0,0.0,0.0,4.0,6.0,1.0,2.0
Cops5,7.0,0.0,NaN,12.0,0.0,10.0,NaN,5.0,13.0,17.0,...,11.0,4.0,2.0,20.0,7.0,0.0,5.0,6.0,NaN,0.0
Arfgef1,1.0,7.0,NaN,0.0,NaN,0.0,0.0,2.0,3.0,5.0,...,5.0,2.0,3.0,NaN,5.0,0.0,8.0,7.0,0.0,0.0
Tram1,14.0,3.0,8.0,6.0,18.0,7.0,19.0,16.0,23.0,21.0,...,3.0,6.0,5.0,6.0,5.0,1.0,8.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Exosc7,0.0,13.0,17.0,12.0,62.0,0.0,10.0,25.0,4.0,2.0,...,4.0,0.0,5.0,5.0,1.0,5.0,0.0,0.0,0.0,0.0
Lars2,0.0,NaN,NaN,2.0,2.0,NaN,NaN,1.0,2.0,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Sacm1l,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,5.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
df.loc['Fn1']

1      19.0
2      15.0
3      52.0
4      35.0
5      30.0
       ... 
678     8.0
679     8.0
680    18.0
681     7.0
682    10.0
Name: Fn1, Length: 682, dtype: float64

In [5]:
def lognpdf(x, mu, sigma):
    shape  = sigma
    loc    = 0
    scale  = np.exp(mu)
    return lognorm.pdf(x, shape, loc, scale)

In [28]:
# count = 1
# for index, row in df.iterrows():
index = 'Fn1'
row = df.loc['Fn1']

start_processing = time.time()
print(f"Start Processing gene: {index}")

gene_name = index
data = np.array(row)

gene = data[data>=0]
k = int(0.95*len(gene))
gene = gene[np.argpartition(gene, k)[:k]]
gene = np.sort(gene)

data_mean = np.mean(gene);
data_var = np.var(gene);
data_noise = data_var/(data_mean**2);

statis_data = utils.statisData(gene);

rho = lambda s: np.sqrt(
            np.sum(
                np.log(statis_data/(s))**2
            )
)


f = lambda k: utils.statisGTM(k,4)
N = 700 # the number of particles
T = 5 # number of rounds
epsilon = 1# a sequence of discrepancy acceptance thresholds

#     prior = lambda: np.array([
#                 random.uniform(0,5), # kon ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # ron ~ logU[-1,1] base 10
#                 random.uniform(0,5), # koff ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # roff ~ logU[-1,1] base 10 
#                 random.uniform(0,50), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
#                 1]) # rdeg=1
prior = lambda: np.array([
            5 * np.random.uniform(), # kon ~ U[0,5]
            10**random.uniform(-1,1), # ron ~ logU[-1,1] base 10
            5 * np.random.uniform(), # koff ~ U[0,5]
            10**random.uniform(-1,1), # roff ~ logU[-1,1] base 10 
            100 * np.random.uniform(), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
            1]) # rdeg=1

proposal_sigma = 0.2

proposal = lambda x: np.random.lognormal(x,proposal_sigma)

proposal_pdf = lambda kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior: \
lognpdf(mu_post,np.log(mu_prior),proposal_sigma) * lognpdf(kon_post,np.log(kon_prior),proposal_sigma) \
* lognpdf(ron_post,np.log(ron_prior),proposal_sigma) * lognpdf(koff_post,np.log(koff_prior),proposal_sigma) \
* lognpdf(roff_post,np.log(roff_prior),proposal_sigma) 


#     result, flag = abcSampler.ABCRejectionSampler(N,prior,f,rho,epsilon,T,count)
#     result = np.load('data/gene_1_prior.npy', allow_pickle='True')

#     result_true = np.load('data/gene_1_prior.npy', allow_pickle='True')

#     print(result_true)

result, flag = abcSampler.ABCSMCSampler(N,prior,f,rho,epsilon,T,proposal,proposal_pdf,gene_name)


# Save posterior
np.save(f'data/test/gene_{gene_name}_cast_N{N}_T{T}.npy', np.array([gene_name, gene, result], dtype=object))

end_processing = time.time()
total_processing = end_processing - start_processing
print(f"Finish Processing gene: {gene_name}, N= {N}, T= {T}, time= {total_processing}")

Start Processing gene: Fn1
Finish Processing gene: Fn1, N= 700, T= 5, time= 6188.706908226013


In [7]:
# data_var

In [8]:
# data_noise

In [9]:
# result[0,0]

In [10]:
# kon_post = 5.30753304760157;
# kon_prior = 3.152078539638027;
# ron_post = 8.70731191726281;
# ron_prior = 9.601789150560395;
# koff_post = 3.5623878531212596;
# koff_prior = 1.1835791487917824;
# roff_post = 4.520511228784241;
# roff_prior = 2.138386913508971;
# mu_post = 101.10914066538857;
# # mu_prior = 95.27428156492888;

# proposal_pdf(kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior) 

In [11]:
# 5 * np.random.uniform()